This notebook is used to initialize the directory structure for the materials directory using an example YAML (or json) file of a Materials Project document.

In [10]:
import yaml
import os
import shutil
import json

with open("mp-188.yaml") as f:
    doc = yaml.load(f)

In [11]:
example = """## Example output

```json
%s
```

"""

def make_dir(d, rootdir):
    for k, v in d.items():
        if k == "_id":
            # Ignore object id key, which is useless.
            continue
        subpath = os.path.join(rootdir, k)
        try:
            os.makedirs(subpath)
        except:
            pass
        fname = os.path.join(subpath, "README.md")
        print fname
        if not os.path.exists(fname):
            with open(fname, "w") as f:
                if v is not None:
                    data = example % json.dumps(v)
                else:
                    data = ""
                f.write(data)
        else:
            with open(fname) as f:
                data = f.read().strip()
            if v is not None:
                data += "\n\n"
                data += example % json.dumps(v)
            with open(fname, "w") as f:
                print "Modifying %s" % k
                f.write(data)
        
        if isinstance(v, dict) and "formula" not in k:
            # Ignore formulas which has material specific keys.
            make_dir(v, rootdir=subpath)

In [12]:
make_dir(doc, rootdir="../materials")

../materials/icsd_ids/README.md


TypeError: dump() takes at least 2 arguments (1 given)